In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv('action_data.csv')
data.head()

,sequence_id,frame,label,kpt_0,kpt_1,kpt_2,kpt_3,kpt_4,kpt_5,kpt_6,...,kpt_24,kpt_25,kpt_26,kpt_27,kpt_28,kpt_29,kpt_30,kpt_31,kpt_32,kpt_33
0,Walking_walking_2.mp4,0,0,0.503828,0.470310,0.502474,0.456438,0.503863,0.454633,0.480826,...,0.506384,0.688987,0.467091,0.809379,0.508653,0.813118,0.465861,0.932251,0.503798,0.942047
1,Walking_walking_2.mp4,1,0,0.530068,0.477002,0.529772,0.463652,0.530697,0.459866,0.513478,...,0.546766,0.689634,0.511066,0.811608,0.538761,0.806040,0.515163,0.918041,0.530440,0.913139
2,Walking_walking_2.mp4,2,0,0.531815,0.472564,0.532459,0.459303,0.532915,0.456127,0.521752,...,0.551426,0.695654,0.510359,0.811137,0.541439,0.809391,0.514119,0.918870,0.535762,0.912767
3,Walking_walking_2.mp4,3,0,0.520036,0.464746,0.520518,0.451350,0.521440,0.448773,0.509508,...,0.538154,0.681027,0.505042,0.799505,0.536438,0.800036,0.505716,0.918724,0.526091,0.920518
4,Walking_walking_2.mp4,4,0,0.517137,0.450228,0.515431,0.437520,0.516919,0.434975,0.496901,...,0.523508,0.666777,0.488468,0.787686,0.519742,0.794045,0.488965,0.911743,0.510291,0.918240


## Dataset Preperation

In [4]:
class ActionCSVDataset(Dataset):
    def __init__(self, csv_file, seq_len=30):
        df = pd.read_csv(csv_file)
        self.seq_len = seq_len
        # Group by sequence_id to keep frames together
        self.sequences = []
        self.labels = []
        
        for _, group in df.groupby('sequence_id'):
            # Extract only the keypoint columns (kpt_0 to kpt_33)
            kpts = group.filter(like='kpt_').values
            if len(kpts) == seq_len:
                self.sequences.append(kpts)
                self.labels.append(group['label'].iloc[0])
        
        self.sequences = torch.tensor(self.sequences, dtype=torch.float32)
        self.labels = torch.tensor(self.labels, dtype=torch.long)

    def __len__(self): return len(self.labels)
    def __getitem__(self, idx): return self.sequences[idx], self.labels[idx]

# --- Training Setup ---
dataset = ActionCSVDataset('action_data.csv')
train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.1)
train_loader = DataLoader(torch.utils.data.Subset(dataset, train_idx), batch_size=16, shuffle=True)

## Bi-LSTM Model

In [5]:
class BiLSTMActionModel(nn.Module):
    def __init__(self, input_size=34, hidden_size=64, num_layers=2, num_classes=2):
        super(BiLSTMActionModel, self).__init__()
        
        # Bi-LSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, bidirectional=True, dropout=0.2)
        
        # Fully connected layer (Hidden * 2 because it's Bidirectional)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, num_classes)
        )
        
    def forward(self, x):
        # x shape: (batch, sequence_length, input_size)
        out, _ = self.lstm(x)
        
        # We only take the output from the last time step
        out = self.fc(out[:, -1, :])
        return out

## Model Training

In [9]:
# Define Model (using the BiLSTMActionModel from before)
model = BiLSTMActionModel(input_size=34, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(150):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")

torch.save(model.state_dict(), 'action_model.pth')

Epoch 1 Loss: 0.7452
Epoch 2 Loss: 0.7484
Epoch 3 Loss: 0.7401
Epoch 4 Loss: 0.7377
Epoch 5 Loss: 0.7362
Epoch 6 Loss: 0.7348
Epoch 7 Loss: 0.7328
Epoch 8 Loss: 0.7306
Epoch 9 Loss: 0.7364
Epoch 10 Loss: 0.7361
Epoch 11 Loss: 0.7276
Epoch 12 Loss: 0.7210
Epoch 13 Loss: 0.7198
Epoch 14 Loss: 0.7181
Epoch 15 Loss: 0.7257
Epoch 16 Loss: 0.7315
Epoch 17 Loss: 0.7321
Epoch 18 Loss: 0.7397
Epoch 19 Loss: 0.7031
Epoch 20 Loss: 0.7080
Epoch 21 Loss: 0.7160
Epoch 22 Loss: 0.7094
Epoch 23 Loss: 0.6899
Epoch 24 Loss: 0.7095
Epoch 25 Loss: 0.7159
Epoch 26 Loss: 0.7025
Epoch 27 Loss: 0.6949
Epoch 28 Loss: 0.6716
Epoch 29 Loss: 0.6456
Epoch 30 Loss: 0.6994
Epoch 31 Loss: 0.6921
Epoch 32 Loss: 0.6598
Epoch 33 Loss: 0.6620
Epoch 34 Loss: 0.6300
Epoch 35 Loss: 0.7037
Epoch 36 Loss: 0.6638
Epoch 37 Loss: 0.6691
Epoch 38 Loss: 0.6131
Epoch 39 Loss: 0.5841
Epoch 40 Loss: 0.5960
Epoch 41 Loss: 0.6137
Epoch 42 Loss: 0.5127
Epoch 43 Loss: 0.5234
Epoch 44 Loss: 0.4509
Epoch 45 Loss: 0.5026
Epoch 46 Loss: 0.51

In [11]:
# Create a dummy input that matches your input shape (Batch, Seq, Features)
dummy_input = torch.randn(1, 30, 34)

# Trace the model
traced_model = torch.jit.script(model, dummy_input)

# Save the serialized model
traced_model.save('action_model_jit.pt')

/tmp/ipykernel_89960/3259376625.py:5: FutureWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution()` instead
  traced_model = torch.jit.script(model, dummy_input)
